# Create MoPo RDF from XLSX Files

## Daten verstehen

### Zusammenhang zwischen Excel und PDF Bericht

Die entscheidenden Spalten sind:

- Abschreibung (Ab)
- automatische Lokalisierung (AL)
- manuelle Lokalisierung (ML)
- Begründungstext (Bg)

- Kapitel 1: ML Kapitel 1 (Anhang 2) und Datum leer oder nach 2022 (Begründungstext muss dann immer etwas haben, 16.3317 wohl fehlerhaft)
- Kapitel 2: AL Kapitel 2 (Anhang 2) und ML leer oder ML Kapitel 2 (Anhang 2) (Begründungstext muss da sein, Abschreibedatum kann schon im Berichtsjahr sein 19.3531 Abschreibung beantragt mit Geschäft...) -- Information
- Anhang 1: AL Anhang 1 und ML leer oder ML Anhang 1 (haben alle einen Wert für "Geschäft in Erfüllung des parlamentarischen Vorstosses") oder 
- Anhang 2: Alle aus Kapitel 1 und 2?


### Typen

| Typ                         | Ab    | AL                   | ML                   | Bg   | Bsp     |
| --------------------------- | ----- | -------------------- | -------------------- | ---- | ------- |
| Hängige Geschäfte           | leer  | Anhang 2             | leer                 | leer | 22.4263 |
| Information                 | leer  | Kapitel 2 (Anhang 2) | leer                 | TXT  | 20.4341 |
| Information                 | leer  | egal                 | Kapitel 2 (Anhang 2) | TXT  | 20.4035 |
| Antrag auf Abschreibung     | leer  | egal                 | Kapitel 1 (Anhang 2) | TXT  | 20.3268 |
| Abgeschrieben mit Botschaft | Datum | Anhang 1             | leer                 | TXT  | 21.3664 |
| Abgeschrieben mit Botschaft | Datum | egal                 | Anhang 1             | TXT  | 16.3884 |
| Abgeschrieben mit Botschaft | Datum | Anhang 1             | leer                 | leer | 20.3917 |

Komische Fälle

| Typ                          | Ab   | AL       | ML   | Bg  | Bsp     |
| ---------------------------- | ---- | -------- | ---- | --- | ------- |
| Abgelehnte Abschreibungen??? | leer | Anhang 2 | leer | TXT | 20.3485 |


- Anträge auf Abschreibung: Noch kein Datum in `Abschreibung`, aber Begründungstext
  - Bsp: 20.4341

- Regulär abgeschrieben: Datum in `Abschreibung` und Begründungstext
  - Bsp: 22.3385
- Nicht regulär abgeschrieben: Datum in `Abschreibung` aber kein Begründungstext
  - Bsp: 22.4022

### Datumsspalten

- `Einreichung` haben alle Zeilen
- `Überweisung` haben alle Zeilen
- `Abschreibung` haben nicht alle Zeilen, falls leer kann Begründung vorhanden sein (Antrag auf Abschreibung) oder noch leer (noch hängig)

### Spalten "Lokalisierung im Bericht"

- Werte in der "manuellen" Spalte, übersteuren die "automatischen"
- falls kein Wert in "manuell", dann gilt "automatisch"

### Mögliche Fehler in den Daten

- 20.3128 (im Jahr 2021) hat in allen Sprachen kein Überweisungsdatum, gemäss: https://www.parlament.ch/de/ratsbetrieb/suche-curia-vista/geschaeft?AffairId=20203128 wohl 04.05.2020 --> so eingefügt
- 19.3734 (im Jahr 2022) hat in der manuellen Lokalisierung: Kapitel 1 (Anhang 2), müsste aber wohl leer sein
- 16.3317 (im Jahr 2022) hat keine Begründung

### Fragen

Wie kann eine abgelehnte MoPo Abschreibung erkannt werden?

## Read XLSX for Every Year and Combine Languages

In [10]:
import pandas as pd
import os

# for which years and which languages are the XLSX files available?
years = [2021, 2022, 2023]
langs = ['De', 'Fr', 'It']

# parameters for pd.read_excel()
dtype_dict = {
    'CuriaId': str,
    'Curia ID': str
}
parse_dates = [5, 6, 7]
date_format = '%d.%m.%Y'

data = pd.DataFrame()

for year in years:

    year_data = pd.DataFrame()

    for lang in langs:
        filename = f"../local/data/Export-{year}-{lang}.xlsx"
        if not os.path.exists(filename):
            print(f"File {filename} does not exist.")
            continue
        print(f"Reading {filename}...")
        file_data = pd.read_excel(
            filename, 
            dtype=dtype_dict, 
            parse_dates=parse_dates, 
            date_format=date_format
        )
        if lang == 'De':
            file_data["Excel Jahr"] = year
            year_data = file_data
        elif lang == 'Fr':
            year_data = pd.merge(
                year_data, 
                file_data[['Curia ID','Titre', 'Texte déposé']], 
                left_on='CuriaId', 
                right_on='Curia ID', 
                how='inner')
            year_data.drop(columns=['Curia ID'], inplace=True)
        elif lang == 'It':
            year_data = pd.merge(
                year_data, 
                file_data[['Curia ID','Titolo', 'Testo depositato']], 
                left_on='CuriaId', 
                right_on='Curia ID', 
                how='inner')
            year_data.drop(columns=['Curia ID'], inplace=True)
    
    data = pd.concat([data, year_data])

data.head()

Reading ../local/data/Export-2021-De.xlsx...
Reading ../local/data/Export-2021-Fr.xlsx...
Reading ../local/data/Export-2021-It.xlsx...
Reading ../local/data/Export-2022-De.xlsx...
Reading ../local/data/Export-2022-Fr.xlsx...
Reading ../local/data/Export-2022-It.xlsx...
Reading ../local/data/Export-2023-De.xlsx...
Reading ../local/data/Export-2023-Fr.xlsx...
Reading ../local/data/Export-2023-It.xlsx...


,CuriaId,Vorstossart,Ursprungsrat,Autor,Titel,Einreichungsdatum,Überweisungsdatum,Abschreibungsdatum,Federführendes Departement,Amt/Direktion,...,Geschäft in Erfüllung des parlamentarischen Vorstosses,Begründungstext Deutsch,Begründungstext Französisch,Begründungstext Italienisch,Eingereichter Text,Excel Jahr,Titre,Texte déposé,Titolo,Testo depositato
0,21.4345,Postulat,Ständerat,"Kommission für Wissenschaft, Bildung und Kultu...",Züchtungsverfahren mit Genom-Editierungsmethoden,2021-11-16,2021-12-02,NaT,UVEK,BAFU,...,NaN,NaN,NaN,NaN,Der Bundesrat erstattet dem Parlament innert J...,2021,Procédés de sélection par édition génomique,"Le Conseil fédéral présente au Parlement, dans...",Procedure di selezione con metodi di editing g...,"Il Consiglio federale presenta al Parlamento, ..."
1,21.4219,Postulat,Nationalrat,Marco Romano,Bekämpfung der internationalen organisierten K...,2021-09-30,2021-12-17,NaT,EJPD,fedpol,...,NaN,NaN,NaN,NaN,"Der Bundesrat wird beauftragt, einen Bericht z...",2021,Lutte contre la criminalité internationale org...,Le Conseil fédéral est chargé de présenter un ...,Lotta alla criminalità organizzata internazion...,Il Consiglio federale è incaricato di esaminar...
2,21.4176,Postulat,Nationalrat,Judith Bellaiche,Cyberrisiken im All,2021-09-30,2021-12-17,NaT,VBS,GS-VBS,...,NaN,NaN,NaN,NaN,"Der Bundesrat wird gebeten, eine Auslegeordnun...",2021,Cyberrisques dans l'espace,Le Conseil fédéral est prié d'établir une vue ...,Ciber-rischi nello spazio extra-atmosferico,Il Consiglio federale è invitato ad analizzare...
3,21.4141,Postulat,Nationalrat,Andri Silberschmidt,Evaluation der Gerichtspraxis nach der Revisio...,2021-09-29,2021-12-17,NaT,EJPD,BJ,...,NaN,NaN,NaN,NaN,"Der Bundesrat wird beauftragt, eine Evaluation...",2021,Évaluation de la pratique des tribunaux suite ...,Le Conseil fédéral est chargé d'évaluer la pra...,Valutazione della prassi giudiziaria dopo la r...,Il Consiglio federale è incaricato di valutare...
4,21.4079,Postulat,Nationalrat,Philipp Kutter,Wirkungsüberprüfung der Steuerreform STAF,2021-09-23,2021-12-17,NaT,EFD,ESTV,...,NaN,NaN,NaN,NaN,"Der Bundesrat wird beauftragt, die Umsetzung d...",2021,Analyse des effets de la réforme fiscale RFFA,Le Conseil fédéral est chargé de procéder à un...,Verificare l’efficacia della riforma fiscale RFFA,Il Consiglio federale è incaricato di valutare...


### Combine "Lokalisierungs" Columns

- If `Lokalisierung im Bericht (manuell)` is empty, use `Lokalisierung im Bericht (automatisch)` otherwise take the former

In [11]:
data["Lokalisierung"] = data["Lokalisierung im Bericht (manuell)"].fillna(data["Lokalisierung im Bericht (automatisch)"])

print(data["Lokalisierung"].value_counts())

Lokalisierung
Anhang 2                1136
Kapitel 2 (Anhang 2)     923
Kapitel 1 (Anhang 2)     435
Anhang 1                 104
Name: count, dtype: int64


## Transform Data to RDF

### Create Lookup Tables for Mapping between Department/Office Abbrevation and URI

In [12]:
import requests

def org_lookup_table(level):
    
    if level == "department":

        sparql_query = """

        PREFIX vl: <https://version.link/>
        PREFIX schema: <http://schema.org/>
        SELECT DISTINCT * WHERE {
        
        ?org schema:parentOrganization <https://ld.admin.ch/FC>;
            a vl:Identity;
            schema:alternateName ?abbr.
        
        FILTER(lang(?abbr) = "de")
        }

        """
    elif level == "office":
        
        sparql_query = """

        PREFIX vl: <https://version.link/>
        PREFIX schema: <http://schema.org/>
        SELECT DISTINCT * WHERE {
        
        ?org schema:parentOrganization/schema:parentOrganization <https://ld.admin.ch/FC>;
            a vl:Identity;
            schema:alternateName ?abbr.
        
        FILTER(lang(?abbr) = "de")
        }

        """

    encoded_query = {"query": sparql_query}

    headers = {
        "Content-Type": "application/x-www-form-urlencoded; charset=utf-8",
        "Accept": "application/sparql-results+json" 
    }

    response = requests.post("https://ld.admin.ch/query", 
                             data=encoded_query, 
                             headers=headers)
    
    if response.status_code == 200:

        response.encoding = "utf-8"

        data = response.json()

        data = data["results"]["bindings"]

        data = [{'org': d['org']['value'], 'abbr': d['abbr']['value']} for d in data]

        data = pd.DataFrame(data)

        data.set_index('abbr', inplace=True)

    return data

dep_lookup = org_lookup_table(level = "department")
office_lookup = org_lookup_table(level = "office")
display(dep_lookup)
display(office_lookup)

,org
abbr,
EDA,https://ld.admin.ch/department/I
EDI,https://ld.admin.ch/department/II
EJPD,https://ld.admin.ch/department/III
VBS,https://ld.admin.ch/department/IV
EFD,https://ld.admin.ch/department/V
WBF,https://ld.admin.ch/department/VI
UVEK,https://ld.admin.ch/department/VII
BK,https://ld.admin.ch/FCh


,org
abbr,
GS-EDA,https://ld.admin.ch/office/I.1.1
STS-EDA,https://ld.admin.ch/office/I.1.2
DV,https://ld.admin.ch/office/I.1.4
DEZA,https://ld.admin.ch/office/I.1.5
DR,https://ld.admin.ch/office/I.1.7
...,...
KS,https://ld.admin.ch/ou/10002695
R,https://ld.admin.ch/ou/10010826
BK,https://ld.admin.ch/ou/10010833


### Line by Line Transformation

In [13]:
from rdflib import Graph, URIRef, Literal, Namespace
import re

g = Graph(bind_namespaces="none") # no predefined namespaces (e.g. RDF, RDFS, OWL)

prov = Namespace("http://www.w3.org/ns/prov#")
g.bind("prov", prov)

rdf = Namespace("http://www.w3.org/1999/02/22-rdf-syntax-ns#")
g.bind("rdf", rdf)

schema = Namespace("http://schema.org/")
g.bind("schema", schema)

chpaf = Namespace("https://ch.paf.link/")
g.bind("chpaf", chpaf)

paf = Namespace("https://paf.link/")
g.bind("paf", paf)

dcterm = Namespace("http://purl.org/dc/terms/")
g.bind("dcterm", dcterm)

# define motion and postulate

motion = URIRef("https://ch.paf.link/Motion")
g.add((motion, rdf.type, paf.Affair))
g.add((motion, rdf.type, chpaf.Motion))
g.add((motion, chpaf.termdat, URIRef("https://register.ld.admin.ch/termdat/109123")))
g.add((motion, chpaf.parlApiId, Literal("5", datatype="http://www.w3.org/2001/XMLSchema#integer")))

postulate = URIRef("https://ch.paf.link/Postulate")
g.add((postulate, rdf.type, paf.Affair))
g.add((postulate, rdf.type, chpaf.Postulate))
g.add((postulate, chpaf.termdat, URIRef("https://register.ld.admin.ch/termdat/109124")))
g.add((postulate, chpaf.parlApiId, Literal("6", datatype="http://www.w3.org/2001/XMLSchema#integer")))


activity_catalog = {}

# line by line processing

def mopo(line, g):

    global activity_catalog

    # if the curia id is not yet seen in the triples, make the registration
    if line['CuriaId'] not in activity_catalog:
        
        # registration_activity
        registration_activity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/registration-activity")
        g.add((registration_activity, rdf.type, prov.Activity))
        g.add((registration_activity, paf.activityType, chpaf.Registration))
        g.add((registration_activity, prov.startedAtTime, Literal(line['Überweisungsdatum'].strftime('%Y-%m-%d'), datatype="http://www.w3.org/2001/XMLSchema#date")))

        # registration_entity
        registration_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/registration-entity")
        g.add((registration_entity, rdf.type, prov.Entity))
        g.add((registration_entity, prov.wasGeneratedBy, registration_activity))    
        g.add((registration_entity, schema.identifier, Literal(line['CuriaId'])))
        g.add((registration_entity, paf.affairType, motion if line['Vorstossart'] == "Motion" else postulate))
        g.add((registration_entity, chpaf.sourceCouncil, URIRef("https://politics.ld.admin.ch/council/N" if line['Ursprungsrat'] == "Nationalrat" else "https://politics.ld.admin.ch/council/S")))
        g.add((registration_entity, schema.author, Literal(line['Autor']))) # author is modelled in entity because we start with registration activity
        g.add((registration_entity, schema.title, Literal(line['Titel'], lang='de')))
        g.add((registration_entity, schema.title, Literal(line['Titre'], lang='fr')))
        g.add((registration_entity, schema.title, Literal(line['Titolo'], lang='it')))
        g.add((registration_entity, schema.description, Literal(line['Eingereichter Text'], lang='de')))
        g.add((registration_entity, schema.description, Literal(line['Texte déposé'], lang='fr')))
        g.add((registration_entity, schema.description, Literal(line['Testo depositato'], lang='it')))
        g.add((registration_entity, chpaf.department, URIRef(dep_lookup.loc[line['Federführendes Departement'], 'org']) if line['Federführendes Departement'] in dep_lookup.index else Literal(line['Federführendes Departement'])))
        g.add((registration_entity, chpaf.office, URIRef(office_lookup.loc[line['Amt/Direktion'], 'org']) if line['Amt/Direktion'] in office_lookup.index else Literal(line['Amt/Direktion'])))
        
        # update the activity catalog
        activity_catalog[line['CuriaId']] = {
            "registration": [line['Excel Jahr']],
            "last_activity": [registration_activity]
        }

    # if there is already an entry for this curia, check whether there was a change in the registration data
    else:

        most_recent_registration_year = activity_catalog[line['CuriaId']]["registration"][-1]

        columns_to_compare = ['Vorstossart', 'Ursprungsrat', 'Autor', 'Titel', 'Einreichungsdatum', 'Überweisungsdatum', 'Federführendes Departement', 'Amt/Direktion']

        # if there was a change
        if (line[columns_to_compare].to_list() != data.loc[(data['CuriaId'] == line['CuriaId']) & (data['Excel Jahr'] == most_recent_registration_year), columns_to_compare].values.flatten().tolist()):

            # registration_activity
            registration_activity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/registration-activity")
            g.add((registration_activity, rdf.type, prov.Activity))
            g.add((registration_activity, paf.activityType, chpaf.Registration))
            g.add((registration_activity, prov.startedAtTime, Literal(line['Überweisungsdatum'].strftime('%Y-%m-%d'), datatype="http://www.w3.org/2001/XMLSchema#date")))
            
            # if there is an update, the activity wasInformedBy the last activity
            g.add((registration_activity, prov.wasInformedBy, activity_catalog[line['CuriaId']]['last_activity'][-1]))

            # registration_entity
            registration_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/registration-entity")
            g.add((registration_entity, rdf.type, prov.Entity))
            g.add((registration_entity, prov.wasGeneratedBy, registration_activity))    
            g.add((registration_entity, schema.identifier, Literal(line['CuriaId'])))
            g.add((registration_entity, paf.affairType, motion if line['Vorstossart'] == "Motion" else postulate))
            g.add((registration_entity, chpaf.sourceCouncil, URIRef("https://politics.ld.admin.ch/council/N" if line['Ursprungsrat'] == "Nationalrat" else "https://politics.ld.admin.ch/council/S")))
            g.add((registration_entity, schema.author, Literal(line['Autor']))) # author is modelled in entity because we start with registration activity
            g.add((registration_entity, schema.title, Literal(line['Titel'], lang='de')))
            g.add((registration_entity, schema.title, Literal(line['Titre'], lang='fr')))
            g.add((registration_entity, schema.title, Literal(line['Titolo'], lang='it')))
            g.add((registration_entity, schema.description, Literal(line['Eingereichter Text'], lang='de')))
            g.add((registration_entity, schema.description, Literal(line['Texte déposé'], lang='fr')))
            g.add((registration_entity, schema.description, Literal(line['Testo depositato'], lang='it')))
            g.add((registration_entity, chpaf.department, URIRef(dep_lookup.loc[line['Federführendes Departement'], 'org']) if line['Federführendes Departement'] in dep_lookup.index else Literal(line['Federführendes Departement'])))
            g.add((registration_entity, chpaf.office, URIRef(office_lookup.loc[line['Amt/Direktion'], 'org']) if line['Amt/Direktion'] in office_lookup.index else Literal(line['Amt/Direktion'])))

            # update the activity catalog
            activity_catalog[line['CuriaId']]['registration'].append(line['Excel Jahr'])
            activity_catalog[line['CuriaId']]['last_activity'].append(registration_activity)
    
    # chapter 2 MoPo Bericht
    if line['Lokalisierung'] == "Kapitel 2 (Anhang 2)":

        information_creation_activity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/information-creation-activity")

        g.add((information_creation_activity, rdf.type, prov.Activity))
        g.add((information_creation_activity, paf.activityType, chpaf.InformationCreation))
        g.add((information_creation_activity, prov.wasInformedBy, activity_catalog[line['CuriaId']]['last_activity'][-1]))
        
        most_recent_registration_year = activity_catalog[line['CuriaId']]["registration"][-1]

        g.add((information_creation_activity, prov.used, URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{most_recent_registration_year}/registration-entity")))

        # update the activity catalog
        activity_catalog[line['CuriaId']]['last_activity'].append(information_creation_activity)

        # information_entity

        information_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/information-entity")

        g.add((information_entity, rdf.type, prov.Entity))
        g.add((information_entity, prov.wasGeneratedBy, information_creation_activity))
        g.add((information_entity, chpaf.mopoInformation, Literal(line['Begründungstext Deutsch'], lang='de')))
        g.add((information_entity, chpaf.mopoInformation, Literal(line['Begründungstext Französisch'], lang='fr')))
        g.add((information_entity, chpaf.mopoInformation, Literal(line['Begründungstext Italienisch'], lang='it')))
            
        # information_activity

        information_activity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/information-activity")

        g.add((information_activity, rdf.type, prov.Activity))
        g.add((information_activity, paf.activityType, chpaf.Information))
        g.add((information_activity, prov.wasInformedBy, information_creation_activity))
        g.add((information_activity, prov.used, URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{most_recent_registration_year}/registration-entity")))
        g.add((information_activity, prov.used, information_entity))

        # update the activity catalog
        activity_catalog[line['CuriaId']]['last_activity'].append(information_activity)


    # chapter 1 MoPo Bericht
    if line['Lokalisierung'] == "Kapitel 1 (Anhang 2)":
        
        # proposal_creation_activity

        proposal_creation_activity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/proposal-creation-activity")

        g.add((proposal_creation_activity, rdf.type, prov.Activity))
        g.add((proposal_creation_activity, paf.activityType, chpaf.ProposalCreation))
        g.add((proposal_creation_activity, prov.wasInformedBy, activity_catalog[line['CuriaId']]['last_activity'][-1]))

        most_recent_registration_year = activity_catalog[line['CuriaId']]["registration"][-1]

        g.add((proposal_creation_activity, prov.used, URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{most_recent_registration_year}/registration-entity")))

        # update the activity catalog
        activity_catalog[line['CuriaId']]['last_activity'].append(proposal_creation_activity)

        # answer_entity

        answer_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/answer-entity")

        g.add((answer_entity, rdf.type, prov.Entity))
        g.add((answer_entity, prov.wasGeneratedBy, proposal_creation_activity))
        g.add((answer_entity, chpaf.mopoAnswer, Literal(line['Begründungstext Deutsch'], lang='de')))
        g.add((answer_entity, chpaf.mopoAnswer, Literal(line['Begründungstext Französisch'], lang='fr')))
        g.add((answer_entity, chpaf.mopoAnswer, Literal(line['Begründungstext Italienisch'], lang='it')))

        # if there was a postulate report
        if "Postulatsbericht" in str(line["Begründungstext Deutsch"]) or "Bericht" in str(line["Begründungstext Deutsch"]):
            parts = line["CuriaId"].split(".")
            g.add((answer_entity, chpaf.postulateReport, Literal(f"https://www.parlament.ch/centers/eparl/curia/20{parts[0]}/20{parts[0]}{parts[1]}/Bericht BR D.pdf", lang='de')))
            g.add((answer_entity, chpaf.postulateReport, Literal(f"https://www.parlament.ch/centers/eparl/curia/20{parts[0]}/20{parts[0]}{parts[1]}/Bericht BR F.pdf", lang='fr')))

        # if there is a link to the "Amtliche Sammlung" (AS YYYY xyz)
        pattern = r'AS (\d{4}) (\d+)(\D)'
        matches = re.findall(pattern, str(line['Begründungstext Deutsch']))
        for match in matches:
            g.add((answer_entity, chpaf.officialCollection, Literal(f"https://www.fedlex.admin.ch/eli/oc/{match[0]}/{match[1]}/de", lang='de')))
            g.add((answer_entity, chpaf.officialCollection, Literal(f"https://www.fedlex.admin.ch/eli/oc/{match[0]}/{match[1]}/fr", lang='fr')))
            g.add((answer_entity, chpaf.officialCollection, Literal(f"https://www.fedlex.admin.ch/eli/oc/{match[0]}/{match[1]}/it", lang='it')))

        # proposal_entity

        proposal_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/proposal-entity")

        g.add((proposal_entity, rdf.type, prov.Entity))
        g.add((proposal_entity, prov.wasGeneratedBy, proposal_creation_activity))
        g.add((proposal_entity, chpaf.proposal, chpaf.Abandonment))

        # proposal_activity

        proposal_activity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/proposal-activity")

        g.add((proposal_activity, rdf.type, prov.Activity))
        g.add((proposal_activity, paf.activityType, chpaf.Proposal))
        g.add((proposal_activity, prov.wasInformedBy, proposal_creation_activity))

        proposal_activty_qa1 = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/proposal-activity/qa1")

        g.add((proposal_activity, prov.qualifiedAssociation, proposal_activty_qa1))
        g.add((proposal_activty_qa1, rdf.type, prov.Association))
        g.add((proposal_activty_qa1, prov.agent, URIRef("https://ld.admin.ch/FC")))
        g.add((proposal_activty_qa1, prov.hadRole, paf.Submitter))

        proposal_activty_qa2 = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/proposal-activity/qa2")
        g.add((proposal_activity, prov.qualifiedAssociation, proposal_activty_qa2))
        g.add((proposal_activty_qa2, rdf.type, prov.Association))
        g.add((proposal_activty_qa2, prov.agent, URIRef("https://ld.admin.ch/FA")))
        g.add((proposal_activty_qa2, prov.hadRole, paf.Receiver))

        g.add((proposal_activity, prov.used, URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{most_recent_registration_year}/registration-entity")))

        g.add((proposal_activity, prov.used, answer_entity))
        g.add((proposal_activity, prov.used, proposal_entity))

        # update the activity catalog
        activity_catalog[line['CuriaId']]['last_activity'].append(proposal_activity)

        # if MoPo was accepted

        if pd.notna(line['Abschreibungsdatum']):

            # decision_activity

            decision_activity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/decision-activity")

            g.add((decision_activity, rdf.type, prov.Activity))
            g.add((decision_activity, paf.activityType, chpaf.Decision))
            g.add((decision_activity, prov.startedAtTime, Literal(line['Abschreibungsdatum'].strftime('%Y-%m-%d'), datatype="http://www.w3.org/2001/XMLSchema#date")))
            g.add((decision_activity, prov.wasInformedBy, proposal_activity))

            g.add((decision_activity, prov.used, URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{most_recent_registration_year}/registration-entity")))

            g.add((decision_activity, prov.used, answer_entity))
            g.add((decision_activity, prov.used, proposal_entity))
            g.add((decision_activity, paf.proposalActivity, proposal_activity))

            # update the activity catalog
            activity_catalog[line['CuriaId']]['last_activity'].append(decision_activity)

            # decision_entity

            decision_entity = URIRef(f"https://politics.ld.admin.ch/curia/{line['CuriaId']}/{line['Excel Jahr']}/decision-entity")

            g.add((decision_entity, rdf.type, prov.Entity))
            g.add((decision_entity, prov.wasGeneratedBy, decision_activity))
            g.add((decision_entity, rdf.predicate, paf.acceptance))
            g.add((decision_entity, paf.acceptance, paf.Accepted))
    
    return g

data.apply(mopo, args=(g,), axis=1)

def write_ttl(g, filename):
    
    with open("../examples/" + filename + ".ttl", 'w', encoding = "utf-8") as ttl_file:
        ttl_file.write(g.serialize())

write_ttl(g, "mopo")